In [ ]:
Sasha Morrison, 2.1 Submission

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime

# Notes:

You are banned from using loops (`for` or `while` or any other) for this entire workshop!

You shouldn't be using loops almost ever with pandas in any case, so break out of the habit now.

## 1. DataFrame basics


Consider the following Python dictionary `data` and Python list `labels`:

``` python
data = {'animal': ['cat', 'cat', 'snake', 'dog', 'dog', 'cat', 'snake', 'cat', 'dog', 'dog'],
        'age': [2.5, 3, 0.5, np.nan, 5, 2, 4.5, np.nan, 7, 3],
        'visits': [1, 3, 2, 3, 2, 3, 1, 1, 2, 1],
        'priority': ['yes', 'yes', 'no', 'yes', 'no', 'no', 'no', 'yes', 'no', 'no']}

labels = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
```
(This is just some meaningless data I made up with the theme of animals and trips to a vet.)

**1.** Create a DataFrame `df` from this dictionary `data` which has the index `labels`.

**2.** Select only the rows where visits are 3 or more. Which types of animals are these?

**3.** Select the rows where visists are 3 and the animal is a cat

**4.** Calculate the sum of all visits in `df` (i.e. the total number of visits).

**5.** Calculate the mean age for each different animal in `df`.

**6.** Append a new row 'k' to `df` with your choice of values for each column. Then delete that row to return the original DataFrame.



In [2]:
data = data = {'animal': ['cat', 'cat', 'snake', 'dog', 'dog', 'cat', 'snake', 'cat', 'dog', 'dog'],
        'age': [2.5, 3, 0.5, np.nan, 5, 2, 4.5, np.nan, 7, 3],
        'visits': [1, 3, 2, 3, 2, 3, 1, 1, 2, 1],
        'priority': ['yes', 'yes', 'no', 'yes', 'no', 'no', 'no', 'yes', 'no', 'no']}

labels = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
df = pd.DataFrame(data)
df = df.set_index(np.array(labels))
msk = df.visits >= 3
msk2 = df.animal == 'cat'
print('answer 2 \n', df.loc[msk]) # answer 2
print('answer 3 \n', df.loc[msk & msk2]) # answer 3
print('answer 4 \n', df.visits.sum()) # answer 4
print('answer 5 \n', df.groupby(['animal']).mean()) # answer 5

k = pd.Series(['horse', 12, 1, 'no'], index=df.columns)
df = df.append(k, ignore_index=True)
df

answer 2 
   animal  age  visits priority
b    cat  3.0       3      yes
d    dog  NaN       3      yes
f    cat  2.0       3       no
answer 3 
   animal  age  visits priority
b    cat  3.0       3      yes
f    cat  2.0       3       no
answer 4 
 19
answer 5 
         age  visits
animal             
cat     2.5     2.0
dog     5.0     2.0
snake   2.5     1.5


,animal,age,visits,priority
0,cat,2.5,1,yes
1,cat,3.0,3,yes
2,snake,0.5,2,no
3,dog,NaN,3,yes
4,dog,5.0,2,no
5,cat,2.0,3,no
6,snake,4.5,1,no
7,cat,NaN,1,yes
8,dog,7.0,2,no
9,dog,3.0,1,no


# 2.1 Shifty problem

You have a DataFrame `df` with a column 'A' of integers. For example:
```python
df = pd.DataFrame({'A': [1, 2, 2, 3, 4, 5, 5, 5, 6, 7, 7]})
```

How do you filter out rows which contain the same integer as the row immediately above?

You should be left with a column containing the following values:

```python
1, 2, 3, 4, 5, 6, 7
```

### Hint: use the `shift()` method

In [3]:
df = pd.DataFrame({'A': [1, 2, 2, 3, 4, 5, 5, 5, 6, 7, 7]})
# shift vals, mask where unshifted - shifted == 0 
df['shifted min orig'] = df.shift().A - df.A
msk = df['shifted min orig'] != 0
df.loc[msk]['A'] 

0    1
1    2
3    3
4    4
5    5
8    6
9    7
Name: A, dtype: int64

# 2.2 columns sum min

Suppose you have DataFrame with 10 columns of real numbers, for example:

```python
df = pd.DataFrame(np.random.random(size=(5, 10)), columns=list('abcdefghij'))
```
Which column of numbers has the smallest sum? Return that column's label.

In [4]:
df = pd.DataFrame(np.random.random(size=(5, 10)), columns=list('abcdefghij'))
minsum = np.argmin(df.sum())
df.columns[minsum]

'j'

# 2.3 Duplicates

How do you count how many unique rows a DataFrame has (i.e. ignore all rows that are duplicates)?

**hint:** There's a method for to find duplicate rows for you

In [5]:
sum(df.duplicated())

0

# 2.4 Group Values

A DataFrame has a column of groups 'grps' and and column of integer values 'vals': 

```python
df = pd.DataFrame({'grps': list('aaabbcaabcccbbc'), 
                   'vals': [12,345,3,1,45,14,4,52,54,23,235,21,57,3,87]})
```
For each *group*, find the sum of the three greatest values.  You should end up with the answer as follows:
```
grps
a    409
b    156
c    345
```

In [6]:
df = pd.DataFrame({'grps': list('aaabbcaabcccbbc'), 
                   'vals': [12,345,3,1,45,14,4,52,54,23,235,21,57,3,87]})
large = df.groupby('grps')['vals'].nlargest(3)
large.groupby('grps').sum()

grps
a    409
b    156
c    345
Name: vals, dtype: int64

# 3. Cleaning Data

### Making a DataFrame easier to work with

It happens all the time: someone gives you data containing malformed strings, Python, lists and missing data. How do you tidy it up so you can get on with the analysis?

Take this monstrosity as the DataFrame to use in the following puzzles:

```python
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN', 'londON_StockhOlm', 
                               'Budapest_PaRis', 'Brussels_londOn'],
              'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
              'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
                   'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )', 
                               '12. Air France', '"Swiss Air"']})
```

Formatted, it looks like this:

```
            From_To  FlightNumber  RecentDelays              Airline
0      LoNDon_paris       10045.0      [23, 47]               KLM(!)
1      MAdrid_miLAN           NaN            []    <Air France> (12)
2  londON_StockhOlm       10065.0  [24, 43, 87]  (British Airways. )
3    Budapest_PaRis           NaN          [13]       12. Air France
4   Brussels_londOn       10085.0      [67, 32]          "Swiss Air"
```

**1.** Some values in the the **FlightNumber** column are missing (they are `NaN`). These numbers are meant to increase by 10 with each row so 10055 and 10075 need to be put in place. Modify `df` to fill in these missing numbers and make the column an integer column (instead of a float column).

In [7]:
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN', 'londON_StockhOlm', 
                               'Budapest_PaRis', 'Brussels_londOn'],
              'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
              'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
                   'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )', 
                               '12. Air France', '"Swiss Air"']})

# df.interpolate()
msk = df['FlightNumber'].isna()
df['FlightNumber'][msk] = df.FlightNumber.shift() + 10
df

<ipython-input-7-c142f6f3883c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['FlightNumber'][msk] = df.FlightNumber.shift() + 10


,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055.0,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075.0,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


# 3.2 column splitting

The **From\_To** column would be better as two separate columns! Split each string on the underscore delimiter `_` to make two new columns `From` and `To` to your dataframe.

In [8]:
ndf = df.From_To.str.split('_', expand=True)
ndf.columns = ['from', 'to']
df['from'] = ndf['from']
df['to'] = ndf['to']
df['from'] = df['from'].str.capitalize()
df['to'] = df['to'].str.capitalize()
df

,From_To,FlightNumber,RecentDelays,Airline,from,to
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!),London,Paris
1,MAdrid_miLAN,10055.0,[],<Air France> (12),Madrid,Milan
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. ),London,Stockholm
3,Budapest_PaRis,10075.0,[13],12. Air France,Budapest,Paris
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air""",Brussels,London


# 3.3 Clean Text

Make the text in your dataframe:

- From and To columns should be lowercase with only first letter capitalized

- In the **Airline** column, you can see some extra puctuation and symbols have appeared around the airline names. Pull out just the airline name. E.g. `'(British Airways. )'` should become `'British Airways'`.

In [9]:
df['Airline'] = df.Airline.str.extract('([a-zA-Z\s]+)', expand=False).str.strip()
df

,From_To,FlightNumber,RecentDelays,Airline,from,to
0,LoNDon_paris,10045.0,"[23, 47]",KLM,London,Paris
1,MAdrid_miLAN,10055.0,[],Air France,Madrid,Milan
2,londON_StockhOlm,10065.0,"[24, 43, 87]",British Airways,London,Stockholm
3,Budapest_PaRis,10075.0,[13],Air France,Budapest,Paris
4,Brussels_londOn,10085.0,"[67, 32]",Swiss Air,Brussels,London


# Exercise 4.1: Column Splitting

Given the unemployment data in `data/country_total.csv`, split the `month` column into two new columns: a `year` column and a `month` column, both integers

In [10]:
df = pd.read_csv('data/country_total.csv')
df['year'] = (df.month // 1).astype(int)
df['month'] = (round(df.month % 1 * 100)).astype(int)
df

,country,seasonality,month,unemployment,unemployment_rate,year
0,at,nsa,1,171000,4.5,1993
1,at,nsa,2,175000,4.6,1993
2,at,nsa,3,166000,4.4,1993
3,at,nsa,4,157000,4.1,1993
4,at,nsa,5,147000,3.9,1993
...,...,...,...,...,...,...
20791,uk,trend,6,2429000,7.7,2010
20792,uk,trend,7,2422000,7.7,2010
20793,uk,trend,8,2429000,7.7,2010
20794,uk,trend,9,2447000,7.8,2010


# 4.2 Group Statistics

Given the unemployment data in `data/country_sex_age.csv`, give the average unemployment rate for:

- Each gender
- Each Age Group
- Both Together

**HINT:** The `seasonality` column makes it such that the data is repeated for each method of calculating unemployment (`nsa`, `trend`, etc.). Can you ignore this and group over it? Or should you take the average for each?

In [11]:
df = pd.read_csv('data/country_sex_age.csv')
grp = df.groupby(['seasonality', 'sex', 'age_group'])
grp = grp.mean()

# 4.3 Estimating group size

Given that we have the unemployment **rate** as a % of total population, and the number of total unemployed, we can estimate the total population.

Give an estimate of the total population for men and women in each age group.

Does this change depending on the unemployment seasonality calculation method?

In [12]:
grp['est pop'] = grp.unemployment / grp.unemployment_rate * 100
grp

month   unemployment  unemployment_rate  \
seasonality sex age_group                                                  
nsa         f   y25-74     1999.433879  244200.087758           7.539839   
                y_lt25     1999.433879  111623.665350          18.818593   
            m   y25-74     1999.433879  267525.815416           6.201653   
                y_lt25     1999.433879  121616.937253          17.215211   
sa          f   y25-74     1999.424416  275348.154482           7.579982   
                y_lt25     1999.424416  114021.946701          18.323837   
            m   y25-74     1999.424416  302136.810603           6.256909   
                y_lt25     1999.424416  125866.182129          17.067671   
trend       f   y25-74     1999.336505  270308.535179           7.579934   
                y_lt25     1999.336505  112878.171857          18.231025   
            m   y25-74     1999.336505  298051.182238           6.272703   
                y_lt25     1999.336505  124500.720877          17.013327   

                                est pop  
seasonality sex age_group                
nsa         f   y25-74     3.238797e+06  
                y_lt25     5.931563e+05  
            m   y25-74     4.313782e+06  
                y_lt25     7.064505e+05  
sa          f   y25-74     3.632570e+06  
                y_lt25     6.222602e+05  
            m   y25-74     4.828851e+06  
                y_lt25     7.374538e+05  
trend       f   y25-74     3.566107e+06  
                y_lt25     6.191543e+05  
            m   y25-74     4.751559e+06  
                y_lt25     7.317835e+05

# 5.1 Tennis

In `data/tennis.csv` you have games that Roger Federer played against various opponents. Questions:

1. How many games did Federer win?

2. What is Federer's win/loss ratio?

3. Who were Federer's top 5 opponents?

In [13]:
df = pd.read_csv('data/tennis.csv')
df2 = df[['year', 'tournament', 'start date', 'type', 'surface',
       'tournament prize money', 'round',
       'opponent', 'ranking', 'score', 'tournament.1',
       'tournament round', 'time', 'winner',
       'player1 total points total', 'win']]
# df2.groupby('opponent').sum()
# df2['opp win'] = df2.groupby('opponent').win.count() - df2.groupby('opponent').win.sum()
# df2

sum_ = df2.groupby('opponent').win.sum()
count = df2.groupby('opponent').win.count()

frame = { 'Count': count, 'Sum': sum_ } 
result = pd.DataFrame(frame)
result['losses'] = result.Count - result.Sum
result['L rate'] = result.Sum / result.Count
# sum = R.Fed wins, count = total matches played
result.sort_values(['L rate']).head(20)

,Count,Sum,losses,L rate
opponent,,,,
Sergi Bruguera (ESP),1,0,1,0.000000
Kenneth Carlsen (DEN),1,0,1,0.000000
Jan Siemerink (NED),1,0,1,0.000000
Patrick Rafter (AUS),3,0,3,0.000000
Andrea Gaudenzi (ITA),1,0,1,0.000000
Orlin Stanoytchev (BUL),1,0,1,0.000000
Francisco Clavet (ESP),1,0,1,0.000000
James Sekulov (AUS),1,0,1,0.000000
Byron Black (ZIM),1,0,1,0.000000


In [14]:
# 1. 972
# 2. 972:207 (~4.7 wins per loss)
# 3. see above.

# 5.2 Over time

1. What was Federer's best year? In terms of money, and then in terms of number of wins

2. Did Federer get better or worse over time?

In [15]:
def is_num(x):
    out = ''
    nums = [0,1,2,3,4,5,6,7,8,9]
    for e in x:
        try:
            if int(e) in nums:
                out += e
        except ValueError:
            continue
    return int(out)

is_num('$14,400')

14400

In [16]:
df = pd.read_csv('data/tennis.csv')

df = df[['year', 'tournament prize money', 'tournament round', 'win', 'winner']]
fin = df.loc[df['tournament round'] == 'F']
fin = fin.loc[fin['win'] == True]
fin['money'] = fin['tournament prize money'].apply(is_num)
# money won in finals
fin.groupby('year')['money'].sum()

year
1999      14400
2001      54000
2002     540600
2003    3026502
2004    6229377
2005    4733250
2006    7221635
2007    7245735
2008    1819800
2009    2938500
2010    4561045
2011    2579000
2012    3971120
Name: money, dtype: int64

In [17]:
"""
5.2
1. Best Year(Money): 2007 ($7,245,735), Best Year(Wins): 2006 (96)
2. RF improved from 1999-2006 and has since plateau'd
"""
# finals wins
fin.groupby('year')['win'].sum()

year
1999     1
2001     1
2002     3
2003     7
2004    11
2005    11
2006    12
2007     8
2008     4
2009     4
2010     5
2011     4
2012     6
Name: win, dtype: int64

# 5.3 Total money won

In the data, you'll find the `tournament round`, one value of which, `F` indicates the final.

Assuming Federer wins the money in the `tournament prize money` if he wins a final in a tournament, how much money has Federer made in tournaments in this dataset?

In [18]:
fin['money'].sum()

44934964

In [19]:
Drinks Section (Aisle 2)
2x Eska Minieral Water (Lemon)
1x Oasis Pineapple
1x Oasis Tropical Passion
1x Oasis Grapefruit

Breakfast/Spreads (By the eggs)
1x Cinnamon Toast Crunch
1x Raspberry Jam

Canned (By the veg/fruit section)
2x Tuna Cans

Dairy Section
3.25% Milk, 2L
Pillbursy Cookie Dough tube

Fruit
6x Lemon
1x Canary Melon (Yellow, on special)

Veg
3x Ginger

Frozen Goods
2x Ilios Frozen Pizza (1x Marg, 1x Four Cheese)
2x Sunland Potsticker Dumplings (1x Beef, 1x Pork)
1x Eggo Waffles (Choc Chip if avail)
1x Breyer's Ice Cream (Heavenly Hash)



SyntaxError: invalid syntax (<ipython-input-19-ed907c4210da>, line 1)